In [1]:
# Imports
import heapq

In [2]:
# Class definition for S-Puzzle
class S_Puzzle:
    def __init__(self, initial_state):
        # Assume input initial_state is valid
        self.__size = len(initial_state)
        self.set_state(initial_state)
    
    def get_size(self):
        return self.__size
    
    def get_state(self):
        return self.__state
    
    def set_state(self, state):
        # Assume input state is valid, and same size as self.__size
        self.__state = state
        
    def get_current_state_children(self):
        children = []
        n = self.__size
        state = self.__state
        
        # Horizontal swaps
        for row in range(n):
            row_tuple = self.__state[row]
            for col in range(n-1):
                new_row = row_tuple[:col] + (row_tuple[col+1],) + (row_tuple[col],) + row_tuple[col+2:]
                new_tuple = state[:row] + (new_row,) + state[row+1:]
                children.append(new_tuple)
        
        # Vertical swaps
        for row in range(n-1):
            row1 = state[row]
            row2 = state[row+1]
            for col in range(n):
                new_row1 = row1[:col] + (row2[col],) + row1[col+1:]
                new_row2 = row2[:col] + (row1[col],) + row2[col+1:]
                new_tuple = state[:row] + (new_row1,) + (new_row2,) + state[row+2:]
                children.append(new_tuple)
        
        return children
    
    def is_current_state_goal(self):
        n = self.__size
        prev = 0
        for row in range(n):
            for col in range(n):
                if self.__state[row][col] != prev + 1:
                    return False
                prev += 1
        return True

In [3]:
# DFS Solver class definition
class DFS_Solver:
    def solve(self, puzzle):
        self.__seen_set = set()
        self.__solution_stack = []
        self.__search_stack = []
        success = self.__solve_dfs(puzzle)
        
        return success, self.__solution_stack, self.__search_stack
        
    def __solve_dfs(self, puzzle):
        state = puzzle.get_state()
        
        self.__search_stack.append(state)
        self.__solution_stack.append(state)
        self.__seen_set.add(state)
        
        if puzzle.is_current_state_goal():
            return True
        
        children = puzzle.get_current_state_children()
        
        for child in children:
            if child in self.__seen_set:
                continue
            
            puzzle.set_state(child)
            success = self.__solve_dfs(puzzle)
            
            if success:
                return True
            
            self.__search_stack.append("Backtrack\n")
        
        self.__solution_stack.pop()
        return False

In [4]:
# Iterative Deepening Solver class definition
class Iterative_Deepening_Solver:
    def solve(self, puzzle):
        initial_state = puzzle.get_state()
        global_search_stack = []
        limit = 0
        while True:
            self.__seen_set = set()
            self.__solution_stack = []
            self.__search_stack = []
            self.__max_depth_seen = 0
            puzzle.set_state(initial_state)
            success = self.__solve_dfs(puzzle, 0, limit)
            
            global_search_stack.append(self.__search_stack)
            
            if success or self.__max_depth_seen < limit:
                break
            
            limit += 1
        
        return success, self.__solution_stack, global_search_stack
        
    def __solve_dfs(self, puzzle, current_depth, limit):
        if current_depth > self.__max_depth_seen:
            self.__max_depth_seen = current_depth
        
        state = puzzle.get_state()
        
        self.__search_stack.append(state)
        self.__solution_stack.append(state)
        self.__seen_set.add(state)
        
        if puzzle.is_current_state_goal():
            return True
        
        children = puzzle.get_current_state_children()
        
        new_depth = current_depth + 1
        
        if new_depth <= limit:
            for child in children:
                if child in self.__seen_set:
                    continue

                puzzle.set_state(child)
                success = self.__solve_dfs(puzzle, current_depth + 1, limit)

                if success:
                    return True

                self.__search_stack.append("Backtrack\n")
        
        self.__solution_stack.pop()
        return False

In [5]:
# A* Solver Class Definition
class A_Star_Solver:
    def __init__(self, heuristic):
        self.__heuristic = heuristic
    
    def solve(self, puzzle):
        # Min heap
        # Elements of heap will be tuples of the form:
        # ( cost + heuristic, cost, state )
        # Where heuristic should be an estimate of the cost to reach goal from state
        # By default, heappush and heappop will compare by first element of tuple
        heap = []
        parents = dict()
        search = []
        seen = set()
        success = False
        
        state = puzzle.get_state()
        
        heapq.heappush(heap, (0, 0, None, state))
        
        while heap:
            priority, cost, parent, state = heapq.heappop(heap)
            
            if state in seen:
                continue
            
            puzzle.set_state(state)
            search.append(state)
            seen.add(state)
            parents[state] = parent
            
            if puzzle.is_current_state_goal():
                success = True
                break
            
            children = puzzle.get_current_state_children()
            
            new_cost = cost + 1
            for child in children:
                if child in seen:
                    continue
                heapq.heappush(heap, (new_cost + self.__heuristic(child), new_cost, state, child))
        
        solution = []
        if success:
            while state is not None:
                solution.append(state)
                state = parents[state]
            solution.reverse()
            
        return success, solution, search
                

In [6]:
# Helper functions

# Function to nicely display a puzzle
def display_puzzle_state(state):
    size = len(state)
    for i in range(size):
        print(state[i])
    print()

# Function to display results of solvers
def print_results(success, solution_path, search_path):
    print('Success:', success)
    print("=================================")
    print('Solution path:')
    for state in solution_path:
        display_puzzle_state(state)
    print("=================================")
    print('Search path:')
    
    for i in range(len(search_path)):
        item = search_path[i]
        if isinstance(item, str):
            print(item)
        elif isinstance(item, tuple):
            display_puzzle_state(item)
        elif isinstance(item, list):
            # In iterative deepening, the search_path is separated into each limits
            print("================")
            print('Limit:', i)
            for state in item:
                if isinstance(state, str):
                    print(state)
                else:
                    display_puzzle_state(state)
                

In [7]:
%%time

# DFS Solver

puzzle = S_Puzzle( ( (2,3), (4,1) ) )
dfs_solver = DFS_Solver()

print_results(*dfs_solver.solve(puzzle))

Success: True
Solution path:
(2, 3)
(4, 1)

(3, 2)
(4, 1)

(3, 2)
(1, 4)

(2, 3)
(1, 4)

(1, 3)
(2, 4)

(3, 1)
(2, 4)

(3, 1)
(4, 2)

(1, 3)
(4, 2)

(4, 3)
(1, 2)

(3, 4)
(1, 2)

(3, 4)
(2, 1)

(4, 3)
(2, 1)

(4, 1)
(2, 3)

(1, 4)
(2, 3)

(1, 4)
(3, 2)

(4, 1)
(3, 2)

(4, 2)
(3, 1)

(2, 4)
(3, 1)

(2, 4)
(1, 3)

(4, 2)
(1, 3)

(1, 2)
(4, 3)

(2, 1)
(4, 3)

(2, 1)
(3, 4)

(1, 2)
(3, 4)

Search path:
(2, 3)
(4, 1)

(3, 2)
(4, 1)

(3, 2)
(1, 4)

(2, 3)
(1, 4)

(1, 3)
(2, 4)

(3, 1)
(2, 4)

(3, 1)
(4, 2)

(1, 3)
(4, 2)

(4, 3)
(1, 2)

(3, 4)
(1, 2)

(3, 4)
(2, 1)

(4, 3)
(2, 1)

(4, 1)
(2, 3)

(1, 4)
(2, 3)

(1, 4)
(3, 2)

(4, 1)
(3, 2)

(4, 2)
(3, 1)

(2, 4)
(3, 1)

(2, 4)
(1, 3)

(4, 2)
(1, 3)

(1, 2)
(4, 3)

(2, 1)
(4, 3)

(2, 1)
(3, 4)

(1, 2)
(3, 4)

Wall time: 2.99 ms


In [8]:
%%time

# Iterative Deepening Solver

puzzle = S_Puzzle( ((6, 1, 2), (7, 8, 3), (5, 4, 9)) )
id_solver = Iterative_Deepening_Solver()

print_results(*id_solver.solve(puzzle))

Success: True
Solution path:
(6, 1, 2)
(7, 8, 3)
(5, 4, 9)

(1, 6, 2)
(7, 8, 3)
(5, 4, 9)

(1, 2, 6)
(7, 8, 3)
(5, 4, 9)

(1, 2, 6)
(7, 8, 3)
(4, 5, 9)

(1, 2, 6)
(7, 5, 3)
(4, 8, 9)

(1, 2, 3)
(7, 5, 6)
(4, 8, 9)

(1, 2, 3)
(4, 5, 6)
(7, 8, 9)

Search path:
Limit: 0
(6, 1, 2)
(7, 8, 3)
(5, 4, 9)

Limit: 1
(6, 1, 2)
(7, 8, 3)
(5, 4, 9)

(1, 6, 2)
(7, 8, 3)
(5, 4, 9)

Backtrack

(6, 2, 1)
(7, 8, 3)
(5, 4, 9)

Backtrack

(6, 1, 2)
(8, 7, 3)
(5, 4, 9)

Backtrack

(6, 1, 2)
(7, 3, 8)
(5, 4, 9)

Backtrack

(6, 1, 2)
(7, 8, 3)
(4, 5, 9)

Backtrack

(6, 1, 2)
(7, 8, 3)
(5, 9, 4)

Backtrack

(7, 1, 2)
(6, 8, 3)
(5, 4, 9)

Backtrack

(6, 8, 2)
(7, 1, 3)
(5, 4, 9)

Backtrack

(6, 1, 3)
(7, 8, 2)
(5, 4, 9)

Backtrack

(6, 1, 2)
(5, 8, 3)
(7, 4, 9)

Backtrack

(6, 1, 2)
(7, 4, 3)
(5, 8, 9)

Backtrack

(6, 1, 2)
(7, 8, 9)
(5, 4, 3)

Backtrack

Limit: 2
(6, 1, 2)
(7, 8, 3)
(5, 4, 9)

(1, 6, 2)
(7, 8, 3)
(5, 4, 9)

(1, 2, 6)
(7, 8, 3)
(5, 4, 9)

Backtrack

(1, 6, 2)
(8, 7, 3)
(5, 4, 9)

Backtrack

(1


(8, 2, 1)
(6, 7, 9)
(5, 4, 3)

Backtrack

Backtrack

(6, 7, 1)
(8, 2, 3)
(5, 4, 9)

(7, 6, 1)
(8, 2, 3)
(5, 4, 9)

Backtrack

(6, 1, 7)
(8, 2, 3)
(5, 4, 9)

Backtrack

(6, 7, 1)
(2, 8, 3)
(5, 4, 9)

Backtrack

(6, 7, 1)
(8, 3, 2)
(5, 4, 9)

Backtrack

(6, 7, 3)
(8, 2, 1)
(5, 4, 9)

Backtrack

(6, 7, 1)
(5, 2, 3)
(8, 4, 9)

Backtrack

(6, 7, 1)
(8, 4, 3)
(5, 2, 9)

Backtrack

(6, 7, 1)
(8, 2, 9)
(5, 4, 3)

Backtrack

Backtrack

(6, 2, 3)
(8, 7, 1)
(5, 4, 9)

(6, 3, 2)
(8, 7, 1)
(5, 4, 9)

Backtrack

(6, 2, 3)
(7, 8, 1)
(5, 4, 9)

Backtrack

(6, 2, 3)
(8, 1, 7)
(5, 4, 9)

Backtrack

(6, 2, 3)
(5, 7, 1)
(8, 4, 9)

Backtrack

(6, 2, 3)
(8, 4, 1)
(5, 7, 9)

Backtrack

(6, 2, 3)
(8, 7, 9)
(5, 4, 1)

Backtrack

Backtrack

(6, 2, 1)
(5, 7, 3)
(8, 4, 9)

(6, 2, 1)
(7, 5, 3)
(8, 4, 9)

Backtrack

(6, 2, 1)
(5, 7, 3)
(4, 8, 9)

Backtrack

(5, 2, 1)
(6, 7, 3)
(8, 4, 9)

Backtrack

(6, 2, 1)
(5, 4, 3)
(8, 7, 9)

Backtrack

(6, 2, 1)
(5, 7, 9)
(8, 4, 3)

Backtrack

Backtrack

(6, 2, 1)
(8, 4, 3)
(5

(5, 4, 3)

Backtrack

(1, 9, 2)
(7, 6, 8)
(5, 4, 3)

Backtrack

(1, 6, 8)
(7, 9, 2)
(5, 4, 3)

Backtrack

(1, 6, 2)
(5, 9, 8)
(7, 4, 3)

Backtrack

(1, 6, 2)
(7, 4, 8)
(5, 9, 3)

Backtrack

(1, 6, 2)
(7, 9, 3)
(5, 4, 8)

Backtrack

Backtrack

(6, 2, 1)
(7, 9, 8)
(5, 4, 3)

(2, 6, 1)
(7, 9, 8)
(5, 4, 3)

Backtrack

(6, 2, 1)
(9, 7, 8)
(5, 4, 3)

Backtrack

(6, 2, 1)
(7, 9, 8)
(4, 5, 3)

Backtrack

(7, 2, 1)
(6, 9, 8)
(5, 4, 3)

Backtrack

(6, 9, 1)
(7, 2, 8)
(5, 4, 3)

Backtrack

(6, 2, 8)
(7, 9, 1)
(5, 4, 3)

Backtrack

(6, 2, 1)
(5, 9, 8)
(7, 4, 3)

Backtrack

Backtrack

(6, 1, 2)
(9, 7, 8)
(5, 4, 3)

(6, 1, 2)
(9, 8, 7)
(5, 4, 3)

Backtrack

(6, 1, 2)
(9, 7, 8)
(4, 5, 3)

Backtrack

(6, 1, 2)
(9, 7, 8)
(5, 3, 4)

Backtrack

(9, 1, 2)
(6, 7, 8)
(5, 4, 3)

Backtrack

(6, 7, 2)
(9, 1, 8)
(5, 4, 3)

Backtrack

(6, 1, 8)
(9, 7, 2)
(5, 4, 3)

Backtrack

(6, 1, 2)
(5, 7, 8)
(9, 4, 3)

Backtrack

(6, 1, 2)
(9, 4, 8)
(5, 7, 3)

Backtrack

(6, 1, 2)
(9, 7, 3)
(5, 4, 8)

Backtrack

Backtrack

(

(1, 2, 9)
(5, 4, 3)

Backtrack

Backtrack

(7, 2, 3)
(1, 8, 6)
(5, 4, 9)

(7, 3, 2)
(1, 8, 6)
(5, 4, 9)

Backtrack

(7, 2, 3)
(1, 6, 8)
(5, 4, 9)

Backtrack

(7, 2, 3)
(5, 8, 6)
(1, 4, 9)

Backtrack

(7, 2, 3)
(1, 4, 6)
(5, 8, 9)

Backtrack

(7, 2, 3)
(1, 8, 9)
(5, 4, 6)

Backtrack

Backtrack

(7, 2, 6)
(5, 8, 3)
(1, 4, 9)

(7, 2, 6)
(8, 5, 3)
(1, 4, 9)

Backtrack

(7, 2, 6)
(5, 8, 3)
(4, 1, 9)

Backtrack

(5, 2, 6)
(7, 8, 3)
(1, 4, 9)

Backtrack

(7, 2, 6)
(5, 4, 3)
(1, 8, 9)

Backtrack

(7, 2, 6)
(5, 8, 9)
(1, 4, 3)

Backtrack

Backtrack

(7, 2, 6)
(1, 4, 3)
(5, 8, 9)

(7, 2, 6)
(4, 1, 3)
(5, 8, 9)

Backtrack

(7, 2, 6)
(1, 3, 4)
(5, 8, 9)

Backtrack

(7, 2, 6)
(1, 4, 3)
(8, 5, 9)

Backtrack

(7, 2, 6)
(1, 4, 3)
(5, 9, 8)

Backtrack

(7, 4, 6)
(1, 2, 3)
(5, 8, 9)

Backtrack

(7, 2, 6)
(1, 4, 9)
(5, 8, 3)

Backtrack

Backtrack

(7, 2, 6)
(1, 8, 9)
(5, 4, 3)

(7, 2, 6)
(1, 9, 8)
(5, 4, 3)

Backtrack

(7, 2, 6)
(1, 8, 9)
(5, 3, 4)

Backtrack

(7, 2, 9)
(1, 8, 6)
(5, 4, 3)

Backtrack

Ba


(1, 3, 6)
(7, 9, 2)
(5, 8, 4)

Backtrack

(1, 6, 3)
(7, 2, 9)
(5, 8, 4)

Backtrack

(1, 6, 3)
(5, 9, 2)
(7, 8, 4)

Backtrack

(1, 6, 3)
(7, 9, 4)
(5, 8, 2)

Backtrack

Backtrack

(1, 6, 2)
(5, 9, 3)
(7, 8, 4)

(1, 6, 2)
(9, 5, 3)
(7, 8, 4)

Backtrack

(1, 6, 2)
(5, 9, 3)
(8, 7, 4)

Backtrack

(5, 6, 2)
(1, 9, 3)
(7, 8, 4)

Backtrack

(1, 6, 2)
(5, 9, 4)
(7, 8, 3)

Backtrack

Backtrack

(1, 6, 2)
(7, 9, 4)
(5, 8, 3)

(1, 6, 2)
(7, 4, 9)
(5, 8, 3)

Backtrack

(1, 6, 2)
(7, 9, 4)
(5, 3, 8)

Backtrack

(1, 6, 4)
(7, 9, 2)
(5, 8, 3)

Backtrack

Backtrack

Backtrack

Backtrack

(1, 6, 3)
(7, 8, 2)
(5, 4, 9)

(1, 6, 3)
(5, 8, 2)
(7, 4, 9)

(6, 1, 3)
(5, 8, 2)
(7, 4, 9)

(6, 3, 1)
(5, 8, 2)
(7, 4, 9)

Backtrack

(6, 1, 3)
(8, 5, 2)
(7, 4, 9)

Backtrack

(6, 1, 3)
(5, 2, 8)
(7, 4, 9)

Backtrack

(6, 1, 3)
(5, 8, 2)
(4, 7, 9)

Backtrack

(6, 1, 3)
(5, 8, 2)
(7, 9, 4)

Backtrack

(5, 1, 3)
(6, 8, 2)
(7, 4, 9)

Backtrack

(6, 8, 3)
(5, 1, 2)
(7, 4, 9)

Backtrack

(6, 1, 3)
(5, 4, 2)
(7, 8, 9)

Ba

Backtrack

(1, 4, 2)
(7, 8, 3)
(5, 6, 9)

(1, 4, 2)
(8, 7, 3)
(5, 6, 9)

Backtrack

(1, 4, 2)
(7, 8, 3)
(6, 5, 9)

Backtrack

(1, 4, 2)
(7, 8, 3)
(5, 9, 6)

Backtrack

(1, 4, 2)
(7, 8, 9)
(5, 6, 3)

Backtrack

Backtrack

(1, 4, 2)
(7, 6, 9)
(5, 8, 3)

(1, 4, 2)
(7, 9, 6)
(5, 8, 3)

Backtrack

(1, 4, 2)
(7, 6, 9)
(5, 3, 8)

Backtrack

(1, 4, 9)
(7, 6, 2)
(5, 8, 3)

Backtrack

Backtrack

Backtrack

Backtrack

(1, 6, 2)
(7, 8, 9)
(5, 4, 3)

(1, 6, 9)
(7, 8, 2)
(5, 4, 3)

(6, 1, 9)
(7, 8, 2)
(5, 4, 3)

(6, 9, 1)
(7, 8, 2)
(5, 4, 3)

Backtrack

(6, 1, 9)
(7, 2, 8)
(5, 4, 3)

Backtrack

(6, 1, 9)
(7, 8, 2)
(4, 5, 3)

Backtrack

(6, 1, 9)
(7, 8, 2)
(5, 3, 4)

Backtrack

(7, 1, 9)
(6, 8, 2)
(5, 4, 3)

Backtrack

(6, 8, 9)
(7, 1, 2)
(5, 4, 3)

Backtrack

(6, 1, 9)
(5, 8, 2)
(7, 4, 3)

Backtrack

(6, 1, 9)
(7, 8, 3)
(5, 4, 2)

Backtrack

Backtrack

(1, 9, 6)
(7, 8, 2)
(5, 4, 3)

(9, 1, 6)
(7, 8, 2)
(5, 4, 3)

Backtrack

(1, 9, 6)
(8, 7, 2)
(5, 4, 3)

Backtrack

(1, 9, 6)
(7, 2, 8)
(5, 4, 3)

Bac

(4, 2, 3)
(7, 9, 5)

Backtrack

(1, 8, 6)
(7, 9, 3)
(4, 2, 5)

Backtrack

(1, 8, 6)
(7, 2, 5)
(4, 9, 3)

Backtrack

Backtrack

(1, 2, 3)
(7, 8, 6)
(4, 9, 5)

(1, 3, 2)
(7, 8, 6)
(4, 9, 5)

Backtrack

(1, 2, 3)
(7, 6, 8)
(4, 9, 5)

Backtrack

(1, 2, 3)
(7, 8, 6)
(4, 5, 9)

Backtrack

(1, 2, 3)
(4, 8, 6)
(7, 9, 5)

Backtrack

(1, 2, 3)
(7, 9, 6)
(4, 8, 5)

Backtrack

(1, 2, 3)
(7, 8, 5)
(4, 9, 6)

Backtrack

Backtrack

(1, 2, 6)
(4, 8, 3)
(7, 9, 5)

(1, 2, 6)
(8, 4, 3)
(7, 9, 5)

Backtrack

(1, 2, 6)
(4, 8, 3)
(9, 7, 5)

Backtrack

(1, 2, 6)
(4, 8, 3)
(7, 5, 9)

Backtrack

(4, 2, 6)
(1, 8, 3)
(7, 9, 5)

Backtrack

(1, 2, 6)
(4, 9, 3)
(7, 8, 5)

Backtrack

(1, 2, 6)
(4, 8, 5)
(7, 9, 3)

Backtrack

Backtrack

(1, 2, 6)
(7, 9, 3)
(4, 8, 5)

(1, 2, 6)
(9, 7, 3)
(4, 8, 5)

Backtrack

(1, 2, 6)
(7, 3, 9)
(4, 8, 5)

Backtrack

(1, 2, 6)
(7, 9, 3)
(8, 4, 5)

Backtrack

(1, 2, 6)
(7, 9, 3)
(4, 5, 8)

Backtrack

(1, 9, 6)
(7, 2, 3)
(4, 8, 5)

Backtrack

(1, 2, 6)
(7, 9, 5)
(4, 8, 3)

Backtrack

Ba

In [9]:
# Heuristic definitions

def h1(state):
    # Number of tiles out of place
    out_of_place = 0
    current_count = 1
    size = len(state)
    for row in range(size):
        for col in range(size):
            if state[row][col] != current_count:
                out_of_place += 1
            current_count += 1
    return out_of_place

def h2(state):
    # Manhattan distance: sum up all the distances by which tiles are out of place
    sum = 0
    size = len(state)
    for row in range(size):
        for col in range(size):
            num = state[row][col]
            real_row = (num-1) // size
            real_col = (num-1) % size
            manhattan_distance = abs(row-real_row) + abs(col-real_col)
            sum += manhattan_distance
    return sum

In [10]:
%%time

# A* solver (Heuristic h1)

puzzle = S_Puzzle( ((6, 1, 2), (7, 8, 3), (5, 4, 9)) )
a_star = A_Star_Solver(heuristic=h1)

print_results(*a_star.solve(puzzle))

Success: True
Solution path:
(6, 1, 2)
(7, 8, 3)
(5, 4, 9)

(1, 6, 2)
(7, 8, 3)
(5, 4, 9)

(1, 2, 6)
(7, 8, 3)
(5, 4, 9)

(1, 2, 3)
(7, 8, 6)
(5, 4, 9)

(1, 2, 3)
(5, 8, 6)
(7, 4, 9)

(1, 2, 3)
(5, 4, 6)
(7, 8, 9)

(1, 2, 3)
(4, 5, 6)
(7, 8, 9)

Search path:
(6, 1, 2)
(7, 8, 3)
(5, 4, 9)

(1, 6, 2)
(7, 8, 3)
(5, 4, 9)

(6, 1, 2)
(5, 8, 3)
(7, 4, 9)

(6, 1, 2)
(7, 4, 3)
(5, 8, 9)

(6, 1, 3)
(7, 8, 2)
(5, 4, 9)

(6, 2, 1)
(7, 8, 3)
(5, 4, 9)

(1, 2, 6)
(7, 8, 3)
(5, 4, 9)

(1, 2, 3)
(7, 8, 6)
(5, 4, 9)

(1, 2, 3)
(5, 8, 6)
(7, 4, 9)

(1, 2, 3)
(7, 4, 6)
(5, 8, 9)

(1, 2, 3)
(5, 4, 6)
(7, 8, 9)

(1, 2, 3)
(4, 5, 6)
(7, 8, 9)

Wall time: 1.01 ms


In [11]:
%%time

# A* solver (Heuristic h2)

puzzle = S_Puzzle( ((6, 1, 2), (7, 8, 3), (5, 4, 9)) )
a_star = A_Star_Solver(heuristic=h2)

print_results(*a_star.solve(puzzle))

Success: True
Solution path:
(6, 1, 2)
(7, 8, 3)
(5, 4, 9)

(1, 6, 2)
(7, 8, 3)
(5, 4, 9)

(1, 2, 6)
(7, 8, 3)
(5, 4, 9)

(1, 2, 3)
(7, 8, 6)
(5, 4, 9)

(1, 2, 3)
(5, 8, 6)
(7, 4, 9)

(1, 2, 3)
(5, 4, 6)
(7, 8, 9)

(1, 2, 3)
(4, 5, 6)
(7, 8, 9)

Search path:
(6, 1, 2)
(7, 8, 3)
(5, 4, 9)

(1, 6, 2)
(7, 8, 3)
(5, 4, 9)

(1, 2, 6)
(7, 8, 3)
(5, 4, 9)

(1, 2, 3)
(7, 8, 6)
(5, 4, 9)

(1, 2, 3)
(5, 8, 6)
(7, 4, 9)

(1, 2, 3)
(5, 4, 6)
(7, 8, 9)

(1, 2, 3)
(4, 5, 6)
(7, 8, 9)

Wall time: 2 ms
